In [1]:
using GAMS

In [2]:
using JuMP

In [3]:
using GLPK

In [4]:
using DataStructures 

In [5]:
using DataFrames

In [6]:
using Surrogates

In [7]:
using Flux

In [8]:
using Statistics

# Novel mathematical formulation for the short-term scheduling of batch plants: Example 1

This paper presents a novel mathematical formulation for the short-term scheduling of batch plants. The proposed formulation is based on a continuous time representation and results in a mixed integer linear programming (MILP) problem. It is implemented for a literature example that involves 3 tasks and 3 units (mixer, reactor and purificator). The main process involves a series of processing tasks i.e. a mixing, reaction and purification.


Formulation and Example Source:
Ierapetritou and Floudas (1998) Effective Continuous-Time Formulation for Short-Term Scheduling. 1. Multipurpose Batch Processes. Independent Engineering Chemical Research 37 (4341-4359)

In [9]:
H = 12;
P = 5;

# SETS
Define the sets to be utilised in the MILP
Please task note that if a task can be performed in more than one unit then it is considered a different task for each unit it can be performed in.

In [10]:
#Sets

I = ["t1","t2","t3"];
J = ["j1","j2","j3"];
N = range(1,5, step = 1);
S = ["s1","s2","s3","s4"];

Jt1 = [J[1]];
Jt2 = [J[2]];
Jt3 = [J[3]];

Ij1 = [I[1]];
Ij2 = [I[2]];
Ij3 = [I[3]];

Is1 = [I[1]];
Is2 = I[1:2];
Is3 = I[2:3];
Is4 = [I[3]];

Ip = I;
Jp = J;
Np = N;


# PARAMETERS
The available information for the process is given below i.e. storage capacity, initial available inventory,
batch size and parameters alpha and beta which are the processing time parameters viz. constant and variable terms


In [11]:
#PARAMETERS

vmin = OrderedDict(
   "t1" => 0,
   "t2" => 0,
   "t3" => 0  
)

vmax = OrderedDict(
   "t1" => 100,
   "t2" => 75,
   "t3" => 50 
)

STin = OrderedDict(
     "s1" => 100000,
     "s2" => 0,
     "s3" => 0,
     "s4" => 0)

STmax = OrderedDict(
     "s1" => 100000,
     "s2" => 100,
     "s3" => 100,
     "s4" => 100000
)

alpha = OrderedDict(
"t1" => 3.0,
"t2" => 2.0,
"t3" => 1.0
)

beta = OrderedDict(
"t1" => 0.03,
"t2" => 0.0266,
"t3" => 0.02     
)

price = OrderedDict(
"s1" => 0,
"s2" => 0,
"s3" => 0,
"s4" => 1
)

demand = OrderedDict(
"s1" => 0,
"s2" => 0,
"s3" => 0,
"s4" => 0
)

#rho_table = wsv"""
#i        s1       s2     s3    s4
#t1       -1       +1      0     0
#t2        0       -1     +1     0
#t3        0       0      -1      +1
#"""

rho_table = DataFrame([(I = "t1", s1 = -1, s2 = +1, s3 = 0, s4 = 0 ),
                       (I = "t2", s1 = 0, s2 = -1, s3 = +1, s4 = 0 ),
                       (I = "t3", s1 = 0, s2 = 0, s3 = -1, s4 = +1 )]);
  
rho = OrderedDict( (r[:I],states) => r[Symbol(states)] for r in eachrow(rho_table), states in S);

# MODEL DEFINITION

In [12]:
example1_netprofit = Model(GAMS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GAMS

# VARIABLES

In [13]:
@variables example1_netprofit begin
    w[i in I, n in N], Bin
    y[j in J, n in N], Bin
    bm[i in I, j in J, n in N] >= 0
    d[s in S, n in N] >= 0
    ST[s in S, n in N] >= 0
    Ts[i in I, j in J, n in N] >= 0
    Tf[i in I, j in J, n in N] >= 0



end

# ALLOCATION CONSTRAINTS
The allocation constraints express that at each unit j and at an event point n only one of the tasks that can be performed
in this unit (i.e. i ∈ Ij ) should take place

In [14]:
@constraints example1_netprofit begin
    allocation1[j in J, n in N; j == J[1]],
      sum(w[i,n] for i in Ij1) == y[j,n]
    allocation2[j in J, n in N; j == J[2]],
      sum(w[i,n] for i in Ij2) == y[j,n]  
    allocation3[j in J, n in N; j == J[3]],
      sum(w[i,n] for i in Ij3) == y[j,n]
end

# CAPACITY CONSTRAINTS

The capacity constraints specify the minimum or maximum amount of available material for a task to take place in a unit
Since vmin is zero for all of them, the minimum capacity constraint is not defined

In [15]:
@constraints example1_netprofit begin
    capacity1[i in I, j in Jt1, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity2[i in I, j in Jt2, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity3[i in I, j in Jt3, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
end

# STORAGE CONSTRAINTS
This constraint specifies the maximum storage capacity of each material state

In [16]:
#STORAGE CONSTRAINTS
@constraints example1_netprofit begin
    storage[s in S, n in N],
      ST[s,n] <= STmax[s]
end

# MATERIAL BALANCE: INITIAL CONDITIONS
This first set constraints is specifying the initial amount of each material state at event point 1


In [17]:
#MATERIAL BALANCE INITIAL
@constraints example1_netprofit begin
    matbal1a[s in S, n in N; s == S[1] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is1 for j in Jt1) - d[s,n]
    matbal1b[s in S, n in N; s == S[2] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is2 for j in Jt2) - d[s,n]
    matbal1c[s in S, n in N; s == S[3] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is3 for j in Jt3) - d[s,n]
    matbal1d[s in S, n in N; s == S[4] && n == N[1]],
      ST[s,n] == STin[s] - d[s,n]
    
end

# MATERIAL BALANCE PROCESSED
The second set of material balance performs a mass balnce on each state i.e. the amount of material s consumed at event point n by task i in unit j and the amount of material s produced at event point n-1 by task i in unit j


In [18]:
#MATERIAL BALANCE PROCESSED
@constraints example1_netprofit begin
    matbal2a[s in S, n in 2:length(N); s == S[1]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt1 for i in Is1) - d[s,n]
    matbal2b[s in S, n in 2:length(N); s == S[2]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt2 for i in Is2) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt1 for i in Is2 ) - d[s,n]
    matbal2c[s in S, n in 2:length(N); s == S[3]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt3 for i in Is3) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt2 for i in Is3 ) - d[s,n]
    matbal2d[s in S, n in 2:length(N); s == S[4]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n-1] for j in Jt3 for i in Is4 ) - d[s,n]
    
end

# DURATION CONSTRAINTS
These constraints don't only specify how long task i will take in unit j but also specifies the dependence of the duration
on the amount of material to be processed by task i in unit j.

In [19]:
#DURATION CONSTRAINTS
@constraints example1_netprofit begin
    duration1[i in I, j in Jt1, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration2[i in I, j in Jt2, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration3[i in I, j in Jt3, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]

end

# SEQUENCE CONSTRAINTS

# i. Same Task in Same Unit
The first set of sequence specify that the start of task i at event point n+1 should start after the end of event point n for the same task performed in unit j.


In [20]:
@constraints example1_netprofit begin
    sequence1a[i in I, j in Jt1, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] -  H*(2 - w[i,n]- y[j,n])
    sequence1b[i in I, j in Jt2, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1c[i in I, j in Jt3, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])

    sequence2a[i in I, j in Jt1, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2b[i in I, j in Jt2, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2c[i in I, j in Jt3, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]

    sequence3a[i in I, j in Jt1, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3b[i in I, j in Jt2, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3c[i in I, j in Jt3, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]

end

# ii. Different Task in Same Unit
The following constraints establishes the relationship between the starting time of task i at point n+1 and the end time of task i′ (ip) at event point n when different tasks take place in the same unit.

In [21]:
@constraints example1_netprofit begin
    sequence4a[i in Ij1, ip in Ij1, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4b[i in Ij2, ip in Ij2, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4c[i in Ij3, ip in Ij3, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
   
end

# ii. Different Task in Different Units

When different tasks i and i′ are performed in different units j and j′ but take place one after the other according to the production recipe.These constraints specify the order in which then tasks in each unit should be performed i.e. mixing then reaction then purification.


In [22]:
@constraints example1_netprofit begin
    sequence5a[i in I, ip in I, j in Jt2, jp in J, n in 1:length(N)-1; i == I[2] && ip == I[1] && jp == J[1]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5b[i in I, ip in I, j in Jt3, jp in J, n in 1:length(N)-1; i == I[3] && ip == I[2] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
     
end

# iv. Completion of previous tasks
A task i' (ip) performed in unit j cannot start until task i in unit j is completed


In [23]:
#cOMPLETION OF PREVIOUS TASKS
@constraints example1_netprofit begin
   sequence6a[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence6b[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence6c[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)

   sequence7a[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence7b[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence7c[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)

   sequence8a[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence8b[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence8c[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)

end

# TIME HORIZON CONSTRAINTS
Specify that all tasks should task place within the time horizon


In [24]:
#TIME HORIZON CONSTRAINTS
@constraints example1_netprofit begin
    timehorizon1[i in I, j in J, n in N],
      Ts[i,j,n] <= H
    timehorizon2[i in I, j in J, n in N],
      Tf[i,j,n] <= H
    
end

# OBJECTIVE FUNCTION

## Profit maximization

In [25]:
@objective example1_netprofit Max begin
    sum(price[s]*d[s,n] for n in N for s in S if s == S[4])
end

d[s4,1] + d[s4,2] + d[s4,3] + d[s4,4] + d[s4,5]

In [26]:
optimize!(example1_netprofit);

┌ Info: Updated GAMS model type: UNDEFINED -> MIP
└ @ GAMS C:\Users\glmab\.julia\packages\GAMS\0iAgU\src\MOI_wrapper\solve.jl:63


--- Job moi.gms Start 04/15/21 16:42:14 34.2.0 r6925a71 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\34\gmsprmNT.txt
    C:\Users\glmab\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\glmab\AppData\Local\Temp\gams_jl_3deB8M\moi.gms
    ScrDir C:\Users\glmab\AppData\Local\Temp\gams_jl_3deB8M\225a\
    SysDir C:\GAMS\34\
    CurDir C:\Users\glmab\AppData\Local\Temp\gams_jl_3deB8M\
    LimRow 0
    LimCol 0
    SolPrint 0
    SavePoint 1
    SolveLink 5
    Threads 1
Licensee: GAMS Community License for Grace Mabele        G200511|0002AO-GEN
          University of the Witwatersrand, SOUTH AFRICA               CL174
          C:\GAMS\34\gamslice.txt
          Grace Mabele, 711140@students.wits.ac.za                         
          Community license for demonstration and instructional purposes only
GAMS 34.2.0   Copyright (C) 1987-2021 GAMS Development. All rights reserved
--- Starting compilation
--- moi.gms(783) 3 Mb
--- Starting execution: elap

# Optimal value and schedule

In [27]:
println("Optimal Solution: ", JuMP.objective_value(example1_netprofit))
println("Optimal Schedule: ")
println(JuMP.value.(w))
println(JuMP.value.(bm))
println(JuMP.value.(Ts))
println(JuMP.value.(Tf))


Optimal Solution: 200.0
Optimal Schedule: 
2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["t1", "t2", "t3"]
    Dimension 2, 1:1:5
And data, a 3×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  1.0  0.0
 1.0  1.0  1.0  1.0  0.0
3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, ["t1", "t2", "t3"]
    Dimension 2, ["j1", "j2", "j3"]
    Dimension 3, 1:1:5
And data, a 3×3×5 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0   0.0
 0.0  0.0  75.0
 0.0  0.0  50.0

[:, :, 2] =
 0.0  0.0   0.0
 0.0  0.0  75.0
 0.0  0.0  50.0

[:, :, 3] =
 0.0  0.0   0.0
 0.0  0.0   0.0
 0.0  0.0  50.0

[:, :, 4] =
 0.0  0.0   0.0
 0.0  0.0  50.0
 0.0  0.0  50.0

[:, :, 5] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, ["t1", "t2", "t3"]
    Dimension 2, ["j1", "j2", "j3"]
    Dimension 3, 1:1:5
And data, a 3×3×5 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 